In [1]:
%pip install tensorflow
%pip install scikit-learn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Example Data (Replace with your actual dataset)
data = pd.read_csv('imdb_reviews.csv')

# Preprocessing
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data['review'])
X = tokenizer.texts_to_sequences(data['review'])

X = pad_sequences(X, maxlen=100)
label_encoder = LabelEncoder()

# Convert sentiment labels to 0 and 1
data['sentiment'] = label_encoder.fit_transform(data['sentiment'])
y = np.array(data['sentiment'])
y = to_categorical(y)

# Saving the tokenizer
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [4]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# Model building
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=100))
model.add(LSTM(units=128))
model.add(Dropout(0.2))
model.add(Dense(units=2, activation='softmax'))  # 2 units for binary classification (positive/negative)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Save the model
model.save('sentiment_model.h5')


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.6868 - val_accuracy: 0.0000e+00 - val_loss: 0.7098
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.6667 - loss: 0.6679 - val_accuracy: 0.0000e+00 - val_loss: 0.7336
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.6667 - loss: 0.6611 - val_accuracy: 0.0000e+00 - val_loss: 0.7612
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.6667 - loss: 0.6470 - val_accuracy: 0.0000e+00 - val_loss: 0.7952
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.6667 - loss: 0.6166 - val_accuracy: 0.0000e+00 - val_loss: 0.8397
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.6667 - loss: 0.6036 - val_accuracy: 0.0000e+00 - val_loss: 0.8985
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.6667 - loss: 0.5917 - val_accuracy: 0.0000e+00 - val_loss: 0.9748
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.6667 - loss: 0.5728 - val_accuracy: 

In [5]:
from keras.models import load_model

# Load the tokenizer and model
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

model = load_model('sentiment_model.h5')

# Example new text for prediction
new_text = ['I absolutely love this movie!']
new_text_seq = tokenizer.texts_to_sequences(new_text)
new_text_padded = pad_sequences(new_text_seq, maxlen=100)

# Predict sentiment
prediction = model.predict(new_text_padded)
print('Positive' if np.argmax(prediction) == 1 else 'Negative')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
Positive
